In [102]:
pip install numpy pandas matplotlib ipython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Завдання 1

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. 
При зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [103]:
import urllib.request
import hashlib
import os
from datetime import datetime

# Папка для збереження файлів
FOLDER_NAME = "vhi_data"
os.makedirs(FOLDER_NAME, exist_ok=True)

def get_timestamp():
    """Повертає поточну дату та час у вигляді рядка."""
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def build_filename(region_id):
    """Створює ім’я файлу для збереження."""
    return os.path.join(FOLDER_NAME, f"VHI_region_{region_id}_{get_timestamp()}.csv")

def hash_file(filepath):
    """Обчислює SHA-256 хеш для вказаного файлу."""
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as file:
        for block in iter(lambda: file.read(8192), b''):
            hasher.update(block)
    return hasher.hexdigest()

def is_duplicate(new_file, region_id):
    """Перевіряє, чи вже існує файл з ідентичним вмістом."""
    new_hash = hash_file(new_file)
    for fname in os.listdir(FOLDER_NAME):
        full_path = os.path.join(FOLDER_NAME, fname)
        if fname.endswith(".csv") and f"region_{region_id}_" in fname and full_path != new_file:
            if hash_file(full_path) == new_hash:
                return True
    return False

def download_vhi(region_id):
    """Завантажує VHI-дані для області."""
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?"
        f"country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean"
    )
    file_path = build_filename(region_id)
    try:
        with urllib.request.urlopen(url) as response:
            content_type = response.getheader("Content-Type", "")
            if "text" not in content_type and "csv" not in content_type:
                print(f"[{region_id}] Помилка: невірний формат відповіді ({content_type}).")
                return

            data = response.read()
            with open(file_path, 'wb') as file:
                file.write(data)

        if is_duplicate(file_path, region_id):
            os.remove(file_path)
            print(f"[{region_id}] Дублікат виявлено. Файл не збережено.")
        else:
            print(f"[{region_id}] Новий файл збережено: {os.path.basename(file_path)}")

    except Exception as e:
        print(f"[{region_id}] Помилка завантаження: {e}")

# Завантаження даних для всіх 27 регіонів України
for region in range(1, 28):
    download_vhi(region)


[1] Дублікат виявлено. Файл не збережено.
[2] Дублікат виявлено. Файл не збережено.
[3] Дублікат виявлено. Файл не збережено.
[4] Дублікат виявлено. Файл не збережено.
[5] Дублікат виявлено. Файл не збережено.
[6] Дублікат виявлено. Файл не збережено.
[7] Дублікат виявлено. Файл не збережено.
[8] Дублікат виявлено. Файл не збережено.
[9] Дублікат виявлено. Файл не збережено.
[10] Дублікат виявлено. Файл не збережено.
[11] Дублікат виявлено. Файл не збережено.
[12] Дублікат виявлено. Файл не збережено.
[13] Дублікат виявлено. Файл не збережено.
[14] Дублікат виявлено. Файл не збережено.
[15] Дублікат виявлено. Файл не збережено.
[16] Дублікат виявлено. Файл не збережено.
[17] Дублікат виявлено. Файл не збережено.
[18] Дублікат виявлено. Файл не збережено.
[19] Дублікат виявлено. Файл не збережено.
[20] Дублікат виявлено. Файл не збережено.
[21] Дублікат виявлено. Файл не збережено.
[22] Дублікат виявлено. Файл не збережено.
[23] Дублікат виявлено. Файл не збережено.
[24] Дублікат виявле

# Завдання 2

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [104]:
import pandas as pd
import os
import re

# Папка з CSV-файлами та вихідний файл
FOLDER_NAME = "vhi_data"  # Моя директорія
output_path = "full.csv" # Файл для збереження 

# Імена колонок для зчитування
COLUMN_NAMES = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "PROVINCE_ID"]

# Порожній DataFrame для об'єднання даних
combined_data = pd.DataFrame(columns=COLUMN_NAMES)

# Отримуємо список CSV-файлів, відсортованих за номером області
files = sorted([f for f in os.listdir(FOLDER_NAME) if f.endswith('.csv')], 
               key=lambda x: int(re.findall(r'\d+', x)[0]))

# Обробка кожного файлу
for file_name in files:
    file_path = os.path.join(FOLDER_NAME, file_name)

    try:
        # Отримуємо ID області з назви файлу
        province_id = int(re.findall(r'\d+', file_name)[0])

        # Зчитуємо файл, пропускаючи перші 2 рядки
        df = pd.read_csv(file_path, skiprows=2, names=COLUMN_NAMES)

        # Видаляємо HTML-теги з Year
        df["Year"] = df["Year"].astype(str).str.replace(r'<tt><pre>|</pre></tt>', '', regex=True)

        # pd.to_numeric(..., errors='coerce') замінює некоректні значення на NaN, якщо такі є.
        df["Year"] = pd.to_numeric(df["Year"], errors='coerce')
        df["Week"] = pd.to_numeric(df["Week"], errors='coerce')

        # Видаляє рядки з NaN у Year та Week, щоб уникнути помилок при подальших обчисленнях
        df.dropna(subset=["Year", "Week"], inplace=True)

        # Перетворює Year і Week на цілі числа (int):
        df["Year"] = df["Year"].astype(int)
        df["Week"] = df["Week"].astype(int)

        # Додаємо ID області у колонку "PROVINCE_ID"
        df["PROVINCE_ID"] = province_id

        # Видаляємо рядки з некоректними VHI та NaN
        df = df[df["VHI"] != -1].dropna()

        # Додаємо оброблені дані в загальний DataFrame
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    except Exception as e:
        print(f"Помилка при читанні файлу {file_name}: {e}")

# Зберігаємо результат у файл
combined_data.to_csv(output_path, index=False)

# Виводимо перші 10 рядків
print("Об'єднані дані успішно збережено у", output_path)
print("Перші 10 рядків:")
print(combined_data.head(10))
print("Останні 10 рядків:")
print(combined_data.tail(10))


C:\Users\User\AppData\Local\Temp\ipykernel_7228\455160447.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, df], ignore_index=True)


Об'єднані дані успішно збережено у full.csv
Перші 10 рядків:
   Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
0  1982    1  0.053  260.31  45.01  39.46  42.23           1
1  1982    2  0.054  262.29  46.83  31.75  39.29           1
2  1982    3  0.055  263.82  48.13  27.24  37.68           1
3  1982    4  0.053  265.33  46.09  23.91  35.00           1
4  1982    5  0.050  265.66  41.46  26.65  34.06           1
5  1982    6  0.048  266.55  36.56  29.46  33.01           1
6  1982    7  0.048  267.84  32.17  31.14  31.65           1
7  1982    8  0.050  269.30  30.30  32.50  31.40           1
8  1982    9  0.052  270.75  28.23  35.22  31.73           1
9  1982   10  0.056  272.73  25.25  37.63  31.44           1
Останні 10 рядків:
       Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
59012  2024   43  0.259  281.45  79.71  17.45  48.58          27
59013  2024   44  0.229  279.41  76.74  13.33  45.04          27
59014  2024   45  0.206  278.07  77.64   8.70  43.17  

# Завдання 3

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [105]:
import pandas as pd

def update_province_ids(df):
    
    province_mapping = {
        1: 21,  2: 26,  3: 25,  4: 27,  5: 3,   6: 4,   7: 8,  
        8: 24,  9: 22, 10: 23, 11: 10, 12: 9,  13: 11, 14: 12, 
       15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 
       22: 20, 23: 6,  24: 1,  25: 2,  26: 7,  27: 5
    }

    # Переконуємося, що колонка "PROVINCE_ID" є у DataFrame
    if "PROVINCE_ID" not in df.columns:
        print("Колонка 'PROVINCE_ID' не знайдена у DataFrame!")
        return df

    # Оновлення значень
    df["PROVINCE_ID"] = df["PROVINCE_ID"].map(province_mapping)

    return df

# Оновлення індексів у `combined_data`
combined_data = update_province_ids(combined_data)

# Збереження у файл
output_file = "Updated_Provinces.csv"
combined_data.to_csv(output_file, index=False)

# Вивід результату
print(f"Оновлений файл збережено: {output_file}")
print(combined_data.head(10))

Оновлений файл збережено: Updated_Provinces.csv
   Year Week    SMN     SMT    VCI    TCI    VHI  PROVINCE_ID
0  1982    1  0.053  260.31  45.01  39.46  42.23           21
1  1982    2  0.054  262.29  46.83  31.75  39.29           21
2  1982    3  0.055  263.82  48.13  27.24  37.68           21
3  1982    4  0.053  265.33  46.09  23.91  35.00           21
4  1982    5  0.050  265.66  41.46  26.65  34.06           21
5  1982    6  0.048  266.55  36.56  29.46  33.01           21
6  1982    7  0.048  267.84  32.17  31.14  31.65           21
7  1982    8  0.050  269.30  30.30  32.50  31.40           21
8  1982    9  0.052  270.75  28.23  35.22  31.73           21
9  1982   10  0.056  272.73  25.25  37.63  31.44           21


# Завдання 4

### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

1. Отримати ряд VHI для області за вказаний рік

In [111]:
def get_vhi_for_region_year(df, province_id, year):

    return df[(df["PROVINCE_ID"] == province_id) & (df["Year"] == year)][["Year", "Week", "VHI"]]

vhi_data = get_vhi_for_region_year(combined_data, province_id=12, year=2023)
print(vhi_data)


       Year Week    VHI
30500  2023    1  55.95
30501  2023    2  55.76
30502  2023    3  53.30
30503  2023    4  51.69
30504  2023    5  47.30
30505  2023    6  42.82
30506  2023    7  40.70
30507  2023    8  41.41
30508  2023    9  41.99
30509  2023   10  42.99
30510  2023   11  44.72
30511  2023   12  45.52
30512  2023   13  46.63
30513  2023   14  47.74
30514  2023   15  49.49
30515  2023   16  52.18
30516  2023   17  56.36
30517  2023   18  62.75
30518  2023   19  67.23
30519  2023   20  69.22
30520  2023   21  70.63
30521  2023   22  69.15
30522  2023   23  70.39
30523  2023   24  72.79
30524  2023   25  73.59
30525  2023   26  74.27
30526  2023   27  72.19
30527  2023   28  71.05
30528  2023   29  70.08
30529  2023   30  68.86
30530  2023   31  68.12
30531  2023   32  66.47
30532  2023   33  61.78
30533  2023   34  57.71
30534  2023   35  55.31
30535  2023   36  50.18
30536  2023   37  44.03
30537  2023   38  40.90
30538  2023   39  39.09
30539  2023   40  37.51
30540  2023   41

2. Пошук екстремумів (min, max), середнього, медіани

In [112]:
def get_vhi_statistics(df, province_ids, years):

    # Фільтруємо дані за обраними областями та роками
    filtered_df = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].isin(years))]

    # Групуємо за областю та роком, обчислюємо статистичні показники
    return filtered_df.groupby(["PROVINCE_ID", "Year"])["VHI"].agg(["min", "max", "mean", "median"]).reset_index()

# Виклик функції для областей з ID 5 і 10 у 2000 та 2020 роках
stats = get_vhi_statistics(combined_data, province_ids=[5, 10], years=[2000, 2023])
print(stats)


   PROVINCE_ID  Year    min    max       mean  median
0            5  2000  27.46  66.30  47.882885  47.375
1            5  2023  32.92  64.08  48.268846  47.890
2           10  2000  10.60  61.87  39.758269  35.915
3           10  2023  31.55  65.72  46.665385  46.260


3. Отримати ряд VHI за вказаний діапазон років для вказаних областей

In [113]:
def get_vhi_by_year_range(df, province_ids, start_year, end_year):

    result = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].between(start_year, end_year))]
    
    if result.empty:
        print("Дані для вказаного діапазону відсутні!")
    
    return result[["Year", "Week", "PROVINCE_ID", "VHI"]]

# Використання:
province_ids = [3, 7, 12]  # Введи потрібні області
start_year = 2010  # Початковий рік
end_year = 2023  # Кінцевий рік
vhi_range_data = get_vhi_by_year_range(combined_data, province_ids, start_year, end_year)
print(vhi_range_data)


       Year Week  PROVINCE_ID    VHI
10150  2010    1            3  52.08
10151  2010    2            3  49.75
10152  2010    3            3  48.82
10153  2010    4            3  48.13
10154  2010    5            3  46.79
...     ...  ...          ...    ...
56779  2023   48            7  34.82
56780  2023   49            7  35.22
56781  2023   50            7  35.57
56782  2023   51            7  36.56
56783  2023   52            7  37.99

[2184 rows x 4 columns]


o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

4. Виявити роки, коли посуха торкнулася >20% областей (VHI < 15)

In [114]:
def find_drought_years(df, threshold=15, affected_percentage=0.2):

    province_count = df["PROVINCE_ID"].nunique()  # Загальна кількість областей
    min_affected = int(province_count * affected_percentage)  # Мінімальна кількість уражених областей

    drought_data = df[df["VHI"] < threshold].groupby(["Year"])["PROVINCE_ID"].nunique().reset_index()
    drought_years = drought_data[drought_data["PROVINCE_ID"] >= min_affected]

    if drought_years.empty:
        print("Не знайдено років, коли посуха торкнулася 20% областей.")
        return None

    # Отримуємо детальні дані про області, які потрапили під посуху
    drought_details = df[(df["Year"].isin(drought_years["Year"])) & (df["VHI"] < threshold)]
    
    return drought_details[["Year", "PROVINCE_ID", "VHI"]]

# Використання:
drought_info = find_drought_years(combined_data)
print(drought_info)


       Year  PROVINCE_ID    VHI
949    2000           21  14.64
950    2000           21  11.82
951    2000           21  10.81
952    2000           21  10.68
953    2000           21  12.30
...     ...          ...    ...
55932  2007            7  11.55
55933  2007            7  10.88
55934  2007            7  11.06
55935  2007            7  12.05
55936  2007            7  13.84

[88 rows x 3 columns]
